In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle as pkl
from datetime import datetime
from tqdm import tqdm
from functions5 import *

#generate random spin configuration. 
def initial(L):
    config = 2*np.random.randint(2, size = (L,L)) - 1 #spin은 +1 or -1
    return config


#Monte-Carlo argorithm with metropolis-hastings. 
def mcmh(config, temp):
    L = int(np.sqrt(config.size))
    for i in range(L**2): #this part is needed to satisfy the definition of one MC sweep.
        x = np.random.randint(0,L)
        y = np.random.randint(0,L)
        s = config[x,y]  
        neigh = config[(x+1)%L, y] + config[x,(y+1)%L] + config[(x-1)%L,y] + config[x,(y-1)%L]
        DE = 2 * s * neigh 

        #accept or reject 
        if DE < 0:
            s *= -1 
        elif np.random.rand() < np.exp(- DE*1/temp):
            s *= -1 
        config[x,y] = s
#     return config


#Calculate energy of given configuration
def energy(config):
    L = len(config)
    E = 0
    for i in range(L):
        for j in range(L):
            s = config[i,j]
            neigh = config[(i+1)%L, j] + config[i,(j+1)%L] + config[(i-1)%L,j] + config[i,(j-1)%L] 
            E += -neigh * s
    return E/2

#magnetization
def magnetization(config):
    magnet = np.sum(config)
    return magnet

def ensemble_magnetization(configuration_set):
    L = len(configuration_set)
    mag = 0
    for i in range(L):
        mag += np.sum(configuration_set[i])
    return mag/L

L = 8; #system의 가로or세로 spin의 개수(N = L*L)
equilsteps = 300; #np.power(2,L);?equilibrium으로 만들기 위한 steps
# mcsteps = 1000; #np.power(2,L);? #mcmh를 몇번 iterate할지 결정

dT = 50; #temperature T의 변화 steps
T = np.linspace(2.0, 4.0, dT) #관심있는 온도범위(critical temperature ~ 2.269)
T2 = np.linspace(2.0, 4.0, dT)

# n1, n2  = 1.0/(mcsteps*L*L), 1.0/(mcsteps*mcsteps*L*L) 

def get_sample(L, T):
    E, M = 0, 0;
    config = initial(L);
    for i in range(equilsteps): # spin configuration을 equilbrium state로 만듦.
        mcmh(config, T)

#     Energy = energy(config);
#     Mag = magnetization(config);

    return T, config



In [14]:
def get_sample_mean_std(T):
    data1=[]
    for l in tqdm(range(10000), position=0, leave=True):
        data1.append(get_sample(L, T))
    data=np.array(data1).transpose()[1]

    print(mean_std(data))

In [2]:
T_list=[1.2, 1.5, 1.85, 2.4, 3.4, 6, 24]

In [3]:
data1=[]

for t in T_list:
    for l in tqdm(range(10000)):
        data1.append(get_sample(L, t))

with open('201015configs.pkl' , 'wb') as f:
    pkl.dump(data1, f)

100%|██████████| 10000/10000 [25:07<00:00,  6.63it/s]


In [15]:
get_sample_mean_std(15)

100%|██████████| 10000/10000 [25:16<00:00,  6.59it/s]


(-8.598, 11.496190499465465)


In [21]:
get_sample_mean_std(5)

100%|██████████| 10000/10000 [26:23<00:00,  6.32it/s]


(-27.4036, 12.546788714248759)


In [16]:
get_sample_mean_std(10)

100%|██████████| 10000/10000 [25:17<00:00,  6.59it/s]


(-13.2424, 11.514809691870726)


In [17]:
get_sample_mean_std(100)

100%|██████████| 10000/10000 [25:19<00:00,  6.58it/s]


(-1.4868, 11.363900112197395)


In [18]:
get_sample_mean_std(1000)

100%|██████████| 10000/10000 [25:20<00:00,  6.57it/s]


(-0.1804, 11.319975964638795)


In [19]:
get_sample_mean_std(10000)

100%|██████████| 10000/10000 [25:26<00:00,  6.55it/s]


(0.1436, 11.249612395100554)


In [20]:
get_sample_mean_std(100000)

100%|██████████| 10000/10000 [25:43<00:00,  6.48it/s]


(0.0496, 11.565082785695916)
